In [1]:
import pandas as pd
from pandas import Series, DataFrame

01.Pandas数据结构
- Series
- DataFrame
- 嵌套字典
- 索引对象  

02.主要功能
- 重新索引
- 丢弃指定轴上的项
- 索引、选取和过滤
- 用loc和iloc进行选取
- 整数索引
- 算术运算和数据对齐
- 在算术方法中填充值
- DataFrame和Series之间的运算
- 函数应用和映射
- 排序和排名
- 重复标签的轴索引  

03.描述和汇总统计
- 相关系数与协方差
- 唯一值、值计数以及成员资格
![](http://mmbiz.qpic.cn/mmbiz_jpg/0y9ibmULDTbDalfn5wX4y1zLhzb72ib3J1qYOwzolA4brEUYOWtMJhmEZNjst1HxsMZVoKVBdOQyLX7A6mVB1pZQ/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1)感觉时间过得好快……
# 01.Pandas数据结构
### Series
&emsp;&emsp;Series是一种类似于一维数组的对象，它由一组数据（各种NumPy数据类型）以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单的Series:索引在左边，值在右边。

In [2]:
obj = pd.Series([4, 7, -5, 3]) #可以传入一个list来创建series
obj

0    4
1    7
2   -5
3    3
dtype: int64

&emsp;&emsp;由于我们没有为数据指定索引，于是会自动创建一个0到N-1（N为数据的长度）的整数型索引。你可以通过Series 的values和index属性获取其数组表示形式和索引对象，这也说明，Series的索引可以通过赋值的方式就地修改：

In [5]:
obj.values, obj.index

(array([ 4,  7, -5,  3], dtype=int64), RangeIndex(start=0, stop=4, step=1))

如果我们希望自己指定索引，可以如下输入：

In [7]:
obj2 = pd.Series([4,-7,5,3],index=['d','b','a','c'])
obj2

d    4
b   -7
a    5
c    3
dtype: int64

In [8]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

与普通NumPy数组相比，你可以通过索引的方式选取Series中的单个或一组值:

In [12]:
print('三个：',obj2[['c','a','d']]) #取多个就用索引列表包括一下，单个就直接写索引
print('一个：',obj2['b'])

三个： c    3
a    5
d    4
dtype: int64
一个： -7


In [13]:
obj2[obj2 > 0] #挑选出obj2里大于0的项

d    4
a    5
c    3
dtype: int64

In [14]:
obj2 * 2

d     8
b   -14
a    10
c     6
dtype: int64

In [16]:
import numpy as np
np.exp(obj2)

d     54.598150
b      0.000912
a    148.413159
c     20.085537
dtype: float64

&emsp;&emsp;还可以将Series看成是一个定长的有序字典，因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

In [17]:
'b' in obj2

True

In [18]:
5 in obj2 #因为5不是index，是value

False

如果数据被存放在一个Python字典中，也可以直接通过这个字典来创建Series：

In [19]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

&emsp;&emsp;如果只传入一个字典，则结果Series中的索引就是原字典的键（有序排列）。你可以传入排好序的字典的键以改变顺序：

In [22]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)#sdata是个字典，按照index指示的顺序拍列，缺失值就是NaN
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [23]:
pd.isnull(obj4) #说明California的值是缺失的

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [24]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [25]:
obj3 + obj4 #有一个NaN则运算也是缺失，NaN

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

### DataFrame
&emsp;&emsp;一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型：数值、字符串、布尔值等，既有行索引也有列索引。  
&emsp;&emsp;建DataFrame的办法有很多，最常用的一种是直接传入一个**由等长列表或NumPy数组组成的字典**，结果DataFrame会自动加上索引（跟Series一样），且全部列会被有序排列：

In [31]:
#例子：
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]} #列表都是等长的，构成一个字典
frame = pd.DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002
5,3.2,Nevada,2003


In [32]:
#  对于特别大的DataFrame，head方法会选取前五行
frame.head()

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002


In [33]:
# 如果指定了列序列，则DataFrame的列就会按照指定顺序进行排列：
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


如果传入的列在数据中找不到，就会在结果中产生缺失值：

In [34]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four','five', 'six'])
frame2 #debt列在data中没有对应项，所以是NaN

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [41]:
#获取列可以用索引方式也可以用属性方式
print(frame2['year'])


one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64
one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object


In [42]:
frame2[['year','state']] #索引列表

,year,state
one,2000,Ohio
two,2001,Ohio
three,2002,Ohio
four,2001,Nevada
five,2002,Nevada
six,2003,Nevada


行也可以通过位置或名称的方式进行获取，比如用loc属性

In [46]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

&emsp;&emsp;将列表或数组赋值给某个列时，其长度必须跟DataFrame的长度相匹配。如果赋值的是一个Series，就会精确匹配DataFrame的索引，所有的空位都将被填上缺失值，为不存在的列赋值会创建出一个新列。

In [44]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
val

two    -1.2
four   -1.5
five   -1.7
dtype: float64

In [45]:
frame2['debt'] = val #本来frame2的debt列全是NaN，现在填充了一部分，空缺仍是NaN
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


关键字del用于删除列。
作为del的例子，我先添加一个新的布尔值的列，state是否为'Ohio'：

In [59]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


In [60]:
del frame2['eastern']

In [62]:
frame2.columns #eastern列就被删除了

Index(['year', 'state', 'pop', 'debt'], dtype='object')

### 嵌套字典
如果嵌套字典传给DataFrame，pandas就会被解释为：**外层字典的键作为列，内层键则作为行索引**：

In [63]:
 pop = {'Nevada': {2001: 2.4, 2002: 2.9},'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [64]:
frame3.T #转置

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


In [67]:
#当然如果给定了指定索引，还是按指定索引来，不会用内层字典的键
pd.DataFrame(pop, index=['2016','2017','2018'])

,Nevada,Ohio
2016,NaN,NaN
2017,NaN,NaN
2018,NaN,NaN


In [69]:
# 由Series组成的字典差不多也是一样的用法
pdata = {'Ohio': frame3['Ohio'][:-1],'Nevada': frame3['Nevada'][:2]} #[列索引][行索引]取值
pd.DataFrame(pdata)

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7


![](https://upload-images.jianshu.io/upload_images/7178691-106835b28c0cea5a.png?imageMogr2/auto-orient/)

如果设置了DataFrame的index和columns的name属性，则这些信息也会被显示出来：

In [70]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3 #列的名称就叫state，行的名称就叫year

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


### 索引对象
&emsp;&emsp;索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建Series或DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个Index(注意：**Index对象是不可变的，因此用户不能对其进行修改**)：

In [71]:
obj = pd.Series(range(3), index=['a', 'b', 'c']) #创建一个series
obj.index

Index(['a', 'b', 'c'], dtype='object')

In [72]:
obj.index[1:]

Index(['b', 'c'], dtype='object')

In [73]:
# 不可变可以使Index对象在多个数据结构之间安全共享
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [74]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [75]:
obj2.index is labels #说明没有变化

True

与python的集合不同，pandas的Index可以包含重复的标签，选择重复的标签，会显示所有的结果。
![](https://upload-images.jianshu.io/upload_images/7178691-5499d14f0e2cd639.jpg?imageMogr2/auto-orient/)

# 02.主要功能
&emsp;&emsp;（**操作Series和DataFrame中的数据的基本手段**）
### 重新索引
reindex，其作用是创建一个新对象，它的数据符合新的索引。如下例，用该Series的reindex将会根据新索引进行重排。如果某个索引值当前不存在，就引入缺失值：

In [4]:
import numpy as np
import pandas as pd
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [9]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e']) #按给定顺序重新排，缺失补NaN
obj2 #注意，是对行索引，列的索引我们叫column，不过啫喱水是series，无需考虑那么多

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对于**时间序列**这样的**有序数据**，重新索引时可能需要做一些插值处理。**method选项**即可达到此目的，例如，使用***ffill***可以实现前向值填充：

In [6]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [8]:
obj3.reindex(range(6), method='ffill') #按照给定的索引序列重新排列，方法选择ffill(前向填充)
# 实际上前向填充就是往下一个填充……

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

借助DataFrame，reindex可以修改（行）索引和列。只传递一个序列时，会重新索引结果的行：

In [10]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [11]:
frame.reindex(['a', 'b', 'c', 'd']) #默认对行的索引

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


列可以用columns关键字重新索引：

In [13]:
frame.reindex(columns = ['Texas','Utah','California'])#原来有的保留，没有的补NaN，不要的丢弃

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


![](https://upload-images.jianshu.io/upload_images/7178691-efa3dbd4b83c61ec.jpg?imageMogr2/auto-orient/)  
### 丢弃指定轴上的项  
drop方法返回的是一个在指定轴上删除了指定值的**新对象**（不是视图）：

In [15]:
# 对于Series
obj = pd.Series(np.arange(5), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0
b    1
c    2
d    3
e    4
dtype: int32

In [16]:
new_obj = obj.drop('c')
new_obj

a    0
b    1
d    3
e    4
dtype: int32

In [17]:
# 对于DataFrame，可以删除任意轴上的索引值
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [18]:
# 用标签序列调用drop会从行标签（axis 0）删除值
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [20]:
# 通过传递axis=1或axis='columns'可以删除列的值
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [21]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


In [22]:
# 使用inplace方法就能就地修改Series和dataframe形状而不产生新对象
obj.drop('c', inplace=True)
obj
# 小心使用inplace，它会销毁所有被删除的数据

a    0
b    1
d    3
e    4
dtype: int32

### 索引、选取和过滤
Series索引obj[...]的工作方式类似于NumPy数组的索引，只不过Series的索引值不只是整数

In [23]:
# 对Series
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [24]:
obj['b']

1.0

In [25]:
obj[1] #等效于上面的索引‘b’

1.0

In [28]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [29]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [30]:
obj[obj < 2] # value＜2 的项，过滤

a    0.0
b    1.0
dtype: float64

In [31]:
# ！！！利用标签的切片运算与普通的Python切片运算不同，其末端是包含的
obj['b':'c']

b    1.0
c    2.0
dtype: float64

用一个值或序列对DataFrame进行索引其实就是获取一个或多个&emsp;**列**

In [32]:
# 对DataFrame
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [33]:
data[['three', 'one']] #用一个值或列表对DataFrame进行索引其实就是获取一个或多个列(不能是行)

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [34]:
data[:2] #切片是在每各列里取片段,等效于行索引(特殊),横向选取各列不能切片

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [48]:
data[data['three'] > 5] #把three列里大于5的部分保留了

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [50]:
data[data < 5] = 0 #DataFrame里＜5的值取0，过滤
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### 用loc和iloc进行选取
&emsp;&emsp;使用轴标签（loc）或整数索引（iloc），从DataFrame选择行和列的子集。

In [51]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [52]:
data.loc['Colorado', ['two', 'three']] #loc先是行索引名，再是列名，准确定位所需取的值

two      5
three    6
Name: Colorado, dtype: int32

In [53]:
data.loc[['Ohio','Utah'],['one','three']]

,one,three
Ohio,0,2
Utah,8,10


In [54]:
# iloc和整数进行选取
data.iloc[[2,3], [3, 0, 1]] #前部是行的索引序号，后是列的索引序号

,four,one,two
Utah,11,8,9
New York,15,12,13


In [55]:
data.iloc[2] #一个值就默认是行索引序号

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [56]:
data.loc[:'Utah', 'two']　#支持切片

Ohio        1
Colorado    5
Utah        9
Name: two, dtype: int32

In [57]:
data.iloc[:, :3][data.three > 5] #选取的所有行，前三列里，满足第三列值＞5的部分，过滤

,one,two,three
Colorado,4,5,6
Utah,8,9,10
New York,12,13,14


![](https://upload-images.jianshu.io/upload_images/7178691-64354f2ab777bd8c.png?imageMogr2/auto-orient/)
##### **ix运算符仍然可用，但并不推荐**
### 整数索引
&emsp;&emsp;ser = pd.Series(np.arange(3.));  
&emsp;&emsp;ser[-1]  
这里，pandas可以勉强进行整数索引，但是会导致小bug，能不这样用尽量不用。  
但对于非整数索引，不会产生歧义：

In [59]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

2.0

为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确，请使用loc（标签）或iloc（整数）：

In [61]:
ser = pd.Series(np.arange(3.))
ser.loc[:1] #loc的切片包含末端，因为实际上loc取的是索引的名

0    0.0
1    1.0
dtype: float64

In [62]:
ser.iloc[:1] # iloc的切片不包含末端，因为取的是索引序列号

0    0.0
dtype: float64

### 算术运算和数据对齐
可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。对于有数据库经验的用户，这就像在索引标签上进行自动外连接。

In [63]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],index=['a', 'c', 'e', 'f', 'g'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [64]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [65]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [66]:
# 对于DataFrame，对齐操作会同时发生在行和列上
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [67]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [68]:
df1+df2 #一个缺失，则运算后整个是NaN

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


### 在算术方法中填充值
在对不同索引的对象进行算术运算时，你可能希望当一个对象中某个轴标签在另一个对象中找不到时填充一个特殊值，比如0

In [70]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),columns=list('abcde'))
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [71]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [73]:
df2.loc[1, 'b'] = np.nan #不用0也可以用NaN
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [74]:
# 将它们相加时，没有重叠的位置就会产生NA值
df1+df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [75]:
df1.add(df2, fill_value=0) #叠加，NaN值补0

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [76]:
# 与此类似，在对Series或DataFrame重新索引时，也可以指定一个填充值
df1.reindex(columns=df2.columns, fill_value=0) # 缺失值补0

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


Series和DataFrame的算术方法主要如下  
![](https://upload-images.jianshu.io/upload_images/7178691-16857a1021f98d1f.png?imageMogr2/auto-orient/)  
### DataFrame和Series之间的运算

In [77]:
# 例子
arr = np.arange(12.).reshape((3, 4))
arr

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [79]:
arr - arr[0] # arr[0]是[0,1,2,3]
# 减去arr[0]时，arr每一行都会减，这叫广播broadcasting
# DataFrame和Series也会有类似于这样的机制

array([[ 0.,  0.,  0.,  0.],
       [ 4.,  4.,  4.,  4.],
       [ 8.,  8.,  8.,  8.]])

In [80]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [81]:
series = frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [83]:
frame - series # 也是 frame每一行都减去了series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [85]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2
#如果某个索引值在DataFrame的列或Series的索引中找不到，则补NaN

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


In [86]:
#如果是希望取出列，并在行的方向上广播（横向广播），则要用算术运算方法
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [87]:
frame.sub(series3, axis='index')#axis置index 表示计算方向是列方向，也可以用axis=0

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


### 函数应用和映射
NumPy的ufuncs（元素级数组方法）也可用于操作pandas对象

In [88]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.513252,-1.268387,-0.665158
Ohio,0.634371,-1.454387,-0.628024
Texas,1.138037,0.206872,-1.155625
Oregon,2.016547,-0.419772,-0.062968


In [89]:
np.abs(frame)

,b,d,e
Utah,0.513252,1.268387,0.665158
Ohio,0.634371,1.454387,0.628024
Texas,1.138037,0.206872,1.155625
Oregon,2.016547,0.419772,0.062968


另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上,DataFrame的apply方法即可实现此功能。

In [91]:
f = lambda x: x.max() - x.min()
frame.apply(f)#默认是将函数应用到 列 上，即沿着index轴向下

b    2.529800
d    1.661260
e    1.092657
dtype: float64

In [93]:
frame.apply(f, axis='columns')#沿着columns轴，即应用到行上

Utah      0.755134
Ohio      2.088759
Texas     2.293662
Oregon    2.436319
dtype: float64

元素级的Python函数也是可以用的。假如你想得到frame中各个浮点值的格式化字符串，使用applymap即可：

In [94]:
format = lambda x: '%.2f' % x #即保留2位小数
frame.applymap(format)

,b,d,e
Utah,-0.51,-1.27,-0.67
Ohio,0.63,-1.45,-0.63
Texas,1.14,0.21,-1.16
Oregon,2.02,-0.42,-0.06


### 排序和排名
要对行或列索引进行排序（按字典顺序），可使用sort_index方法，它将返回一个已排序的新对象

In [6]:
import numpy as np
import pandas as pd
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int32

In [8]:
obj.sort_index() #对索引排序后生成新对象

a    1
b    2
c    3
d    0
dtype: int32

对于DataFrame，则可以根据任意一个轴上的索引进行排序：

In [9]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [10]:
frame.sort_index() #默认是对行索引按序排列

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [11]:
frame.sort_index(axis=1)#指定columns按序排列

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [14]:
# 数据默认是按升序排序的，但也可以降序排序
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


若要按值对Series进行排序，可使用其**sort_values**方法：

In [15]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values() #默认也是升序

2   -3
3    2
0    4
1    7
dtype: int64

In [16]:
# 在排序时，任何缺失值默认都会被放到Series的末尾：

obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

当排序一个DataFrame时，你可能希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给sort_values的by选项即可达到该目的

In [17]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,a,b
0,0,4
1,1,7
2,0,-3
3,1,2


In [18]:
frame.sort_values(by='b')

,a,b
2,0,-3
3,1,2
0,0,4
1,1,7


In [19]:
frame.sort_values(by=['a', 'b'])

,a,b
2,0,-3
0,0,4
3,1,2
1,1,7


Series和DataFrame的rank方法。默认情况下，rank是通过“为各组分配一个平均排名”的方式破坏平级关系的。

In [2]:
import pandas as pd
import numpy as np
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

In [3]:
obj.rank() #没看懂

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [5]:
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

排名时用于破坏平级关系的方法
![](https://upload-images.jianshu.io/upload_images/7178691-7edfab5b4a147581.png?imageMogr2/auto-orient/)  
### 重复标签的轴索引
虽然许多pandas函数（如reindex）都要求标签唯一，但这并不是强制性的。如

In [6]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int32

In [7]:
# 索引的is_unique属性可以告诉你它的值是否是唯一的
obj.index.is_unique

False

In [8]:
#对于带有重复值的索引，数据选取的行为将会有些不同
#如果某个索引对应多个值，则返回一个Series；而对应单个值的，则返回一个标量值
obj['a']
#这样会使代码变复杂，因为索引的输出类型会根据标签是否有重复发生变化

a    0
a    1
dtype: int32

In [9]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,0.070422,1.622645,-1.105333
a,-0.692946,0.937494,-1.152876
b,0.710564,-0.080623,-0.254267
b,0.323067,1.885241,-0.710721


In [10]:
df.loc['b']

,0,1,2
b,0.710564,-0.080623,-0.254267
b,0.323067,1.885241,-0.710721


# 03. 描述和汇总统计

In [11]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],[np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [12]:
# 调用DataFrame的sum方法将会返回一个含有列的和的Series（NaN在计算中被当做0）
df.sum()

one    9.25
two   -5.80
dtype: float64

In [13]:
# 传入axis='columns'或axis=1将会按行进行求和运算
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [14]:
#NA值会自动被排除，除非整个切片（这里指的是行或列）都是NA
# 通过skipna选项可以禁用该功能
df.mean(axis='columns', skipna=False) #求每行元素的均值

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

![](https://upload-images.jianshu.io/upload_images/7178691-af35e3809278410e.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/700)  
有些方法（如idxmin和idxmax）返回的是间接统计（比如达到最小值或最大值的索引）：

In [16]:
df.idxmax()

one    b
two    d
dtype: object

In [17]:
# 另一些方法则是累计型的
df.cumsum() #默认沿axis=0累加

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [18]:
# describe汇总统计
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [19]:
# 对于非数值型数据，describe会产生另外一种汇总统计
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

![](https://upload-images.jianshu.io/upload_images/7178691-11fa967f658ac314.jpg?imageMogr2/auto-orient/)  
### 相关系数与协方差
&emsp;&emsp;Series的corr方法用于计算两个Series中重叠的、非NA的、按索引对齐的值的相关系数。与此类似，cov用于计算协方差。a.corr(b), a.cov(b)。  
&emsp;&emsp;另一方面，DataFrame的corr和cov方法将以DataFrame的形式分别返回完整的相关系数或协方差矩阵。  
&emsp;&emsp;利用DataFrame的corrwith方法，你可以计算其列或行跟另一个Series或DataFrame之间的相关系数。传入一个Series将会返回一个相关系数值Series（针对各列进行计算）
### 唯一值、值计数以及成员资格


In [22]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [24]:
#得到Series中的唯一值数组，返回的唯一值是未排序的
uniques = obj.unique();uniques

array(['c', 'a', 'd', 'b'], dtype=object)

In [25]:
#value_counts用于计算一个Series中各值出现的频率
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

In [26]:
#value_counts还是一个顶级pandas方法，可用于任何数组或序列
pd.value_counts(obj.values, sort=False)

c    3
b    2
a    3
d    1
dtype: int64

isin用于判断矢量化集合的成员资格，可用于过滤Series中或DataFrame列中数据的子集

In [27]:
mask = obj.isin(['b', 'c'])
mask #是b或者c则为True，否则为False

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [28]:
obj[mask] #为False的元素都被过滤掉了

0    c
5    b
6    b
7    c
8    c
dtype: object

与isin类似的是Index.get_indexer方法，它可以给你一个索引数组，从可能包含重复值的数组到另一个不同值的数组

In [29]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match) #获取to_match中每个元素在unique_vals中的索引

array([0, 2, 1, 1, 0, 2], dtype=int64)

![](https://upload-images.jianshu.io/upload_images/7178691-b53c4a9d65a2db32.png?imageMogr2/auto-orient/)